In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key = "",
    model_name="llama-3.3-70b-versatile"
)

result = llm.invoke("What is Groq?")
print(result.content)

Groq is a technology company that specializes in the development of artificial intelligence (AI) and machine learning (ML) hardware and software solutions. The company was founded in 2016 by Jonathan Ross and a team of experienced engineers and researchers from Google, Tesla, and other leading technology companies.

Groq's primary focus is on creating high-performance, low-power AI computing platforms that can be used for a wide range of applications, including natural language processing, computer vision, and autonomous vehicles. The company's products include:

1. Tensor Streaming Processor (TSP): A custom-designed AI accelerator chip that provides high-performance processing for ML workloads.
2. GroqCard: A PCIe-based accelerator card that integrates the TSP chip and provides a scalable, high-performance platform for AI computing.
3. GroqWare: A software development kit (SDK) that provides a set of tools and libraries for developing and optimizing AI applications on Groq's hardware 